In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
from read_dataset import build_df
from utils import CFG
from sklearn.model_selection import train_test_split
import torch
from torch.autograd import Variable
from models import *
from main import *
from tqdm import tqdm
from torch.utils.data import DataLoader
from datapreprocess import *

import seaborn as sns
from sklearn.metrics import *

# Parameters
params = {"batch_size": 64, "shuffle": True, "num_workers": 6}
max_epochs = 100
NUM_CLASSES = 11
RESNET_OUT_FEATURES = 1000



IMG_FILES = glob(CFG.img_path + "/*.jpg")
XML_FILES = glob(CFG.xml_path + "/*.xml")
df, classes = build_df(XML_FILES)
data = df.to_numpy()

    # input and target
# input and target
input = df[["img_path"]].values
input = np.squeeze(input)
# input = input.reset_index()
target = df[["xmin", "ymin", "xmax", "ymax", "label"]].values.astype(np.int32)


    # splitting data
    # train_data, validation_data, test_data = split_dataset(input,target,True)
train_df, test_df = split_dataset(input, target, validation=False)


XML_FILES_Size:  11000
IMG_FILES_Size:  11000


In [2]:
train_df[1][4] # location and label of the fourth item

array([372, 444, 520, 540,  10], dtype=int32)

In [3]:
# train_dataset  = ResNetDataset(train_df, transform=None)
# test_dataset = ResNetDataset(test_df, transform=None)
# train_dataset[0]

In [4]:

transform = preprocess_image
train_dataset  = Custom_Dataset(train_df, transform=transform)
test_dataset = Custom_Dataset(test_df, transform=transform)
# train_dataset[0][1]

In [5]:
# type(test_dataset[0][0])

In [6]:
# plt.imshow(test_dataset[0][0].permute(1,2,0))

In [7]:

# Dataloaders
train_dataloader = DataLoader(
    train_dataset, params["batch_size"], num_workers=params["num_workers"], shuffle=params["shuffle"],
)
# validation_dataloader = torch.utils.data.DataLoader(validation_data, params['batch_size'],num_workers=params['num_workers'])
test_dataloader = DataLoader(
    test_dataset, params["batch_size"], num_workers=params["num_workers"]
)


In [8]:

def get_default_device():
    """gets gpu for mac m1 machine"""
    if torch.backends.mps.is_available():
        mps_device = torch.device("mps")
        # x = torch.ones(1, device=mps_device)
        # print(x)
        return mps_device
    else:
        # print("MPS device not found.")
        return torch.device('cpu')

In [9]:

def train_model(train_dataloader, loss_fn,in_channels=1000,num_classes=11, epochs=30, learning_rate=0.001, device=None):
    
    if device is None:
        device = 'cpu'
    model = get_custom_model(res_out_features=in_channels, n_classes=num_classes).to(device)

    optimizer = torch.optim.Adam(
        params=model.parameters(), lr=learning_rate)

    training_losses = []
    training_accs = []

    for epoch in range(1, epochs+1):
        number_of_batches = 0
        epoch_loss_values = 0.0
        epoch_accs = 0.0
        for index, batch in enumerate(tqdm(train_dataloader)):
            # X,  y = X.to(device), y.to(device)  # make the tensors in gpu state
            X   = batch[0]
            y = batch[1]
            X  = Variable(X, requires_grad=True).to(device)
            y  = torch.LongTensor(y).to(device)

            preds = model(X)

            loss = loss_fn(preds, y).to(device)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            probs = torch.log_softmax(preds, dim=1)
            predicted_labels = torch.argmax(probs, dim=1)

            # acc
            epoch_accs += accuracy_score(y.detach().cpu(),
                                         predicted_labels.detach().cpu())
            epoch_loss_values += loss.item()

            number_of_batches += 1

        # compute average of batch loss and accuracy
        batch_acc, batch_loss = epoch_accs / \
            number_of_batches, epoch_loss_values / number_of_batches
        training_losses.append(batch_loss)
        training_accs.append(batch_acc)

        print("Epoch:{}/{}, acc={:.3f}%, loss={:.3f}".format(epoch,
              epochs, batch_acc*100, batch_loss))

    print("Learning Finished!")

    return model, training_accs, training_losses



In [10]:
DEVICE = get_default_device()

loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.001
input_features = RESNET_OUT_FEATURES
EPOCHS = 1

In [11]:
trained_model,training_accs, training_losses =  train_model(train_dataloader, loss_fn, input_features, num_classes = NUM_CLASSES, epochs=EPOCHS, learning_rate=learning_rate,device=DEVICE)

Using cache found in /Users/gere/.cache/torch/hub/pytorch_vision_v0.10.0
  0%|          | 0/137 [00:00<?, ?it/s]

XML_FILES_Size:  11000
IMG_FILES_Size:  11000
XML_FILES_Size:  11000
IMG_FILES_Size:  11000
XML_FILES_Size:  11000
IMG_FILES_Size:  11000
XML_FILES_Size:  11000
IMG_FILES_Size:  11000
XML_FILES_Size:  11000
IMG_FILES_Size:  11000
XML_FILES_Size:  11000
IMG_FILES_Size:  11000


  1%|▏         | 2/137 [02:00<2:15:10, 60.08s/it]

In [ ]:

for batch in tqdm(train_dataloader):
    print(batch[0])
    break

In [ ]:
def evalution_metrics(ground_truth, predictions):
    print(f"mean acc score = {accuracy_score(ground_truth, predictions)}")
    print(f"mean recall score = {recall_score(ground_truth, predictions, average='micro')}")
    print(f"precision score = {precision_score(ground_truth, predictions, average='micro')}")
    print(f"mean f1 score = {f1_score(ground_truth, predictions, average='micro')}")
    labels = np.unique(ground_truth).tolist()
    cm  = confusion_matrix(ground_truth, predictions, labels=labels) 
    report  = classification_report(ground_truth, predictions)
    print(report)

    sns.heatmap(cm)
    plt.show()
